In [ ]:
# !pip install pystac-client sat-search
# !pip install rasterio

In [ ]:
# from pystac_client import Client

# LandsatSTAC = Client.open("https://landsatlook.usgs.gov/stac-server", headers=[])

# for collection in LandsatSTAC.get_collections():
#     print(collection)

In [1]:
import satsearch

SentinelSTAC = satsearch.Search.search( url = "https://earth-search.aws.element84.com/v0" )
print("Found " + str(SentinelSTAC.found()) + "items")

Found 53392130items


In [2]:
from json import load

file_path = "map.geojson"
file_content = load(open(file_path))
geometry = file_content["features"][0]["geometry"]

timeRange = '2019-06-01/2021-06-01'

In [4]:
# LandsatSearch = LandsatSTAC.search ( 
#     intersects = geometry,
#     datetime = timeRange,
#     query =  ['eo:cloud_cover95'],
#     collections = ["landsat-c2l2-sr"] )

# Landsat_items = [i.to_dict() for i in LandsatSearch.get_items()]
# print(f"{len(Landsat_items)} Landsat scenes fetched")

In [6]:
# for item in Landsat_items:
#     red_href = item['assets']['red']['href']
#     red_s3 = item['assets']['red']['alternate']['s3']['href']
#     print(red_href)    
#     print(red_s3)

In [7]:
SentinelSearch = satsearch.Search.search( 
    url = "https://earth-search.aws.element84.com/v0",
    intersects = geometry,
    datetime = timeRange,
    collections = ['sentinel-s2-l2a-cogs'] )

Sentinel_items = SentinelSearch.items()
print(Sentinel_items.summary())

for item in Sentinel_items:
    red_s3 = item.assets['B04']['href']
    print(red_s3)

Items (1417):
date                      id                        
2021-05-30                S2B_37MDV_20210530_0_L2A  
2021-05-28                S2A_37MBV_20210528_0_L2A  
2021-05-28                S2A_37MCV_20210528_0_L2A  
2021-05-28                S2A_37MDV_20210528_0_L2A  
2021-05-28                S2A_37NBA_20210528_0_L2A  
2021-05-28                S2A_37NCA_20210528_0_L2A  
2021-05-28                S2A_37NDA_20210528_0_L2A  
2021-05-28                S2A_37NBB_20210528_0_L2A  
2021-05-28                S2A_37NCB_20210528_0_L2A  
2021-05-28                S2A_37NDB_20210528_0_L2A  
2021-05-25                S2A_37MDV_20210525_0_L2A  
2021-05-23                S2B_37MBV_20210523_0_L2A  
2021-05-23                S2B_37MCV_20210523_0_L2A  
2021-05-23                S2B_37MDV_20210523_0_L2A  
2021-05-23                S2B_37NBA_20210523_0_L2A  
2021-05-23                S2B_37NCA_20210523_0_L2A  
2021-05-23                S2B_37NDA_20210523_0_L2A  
2021-05-23                S2B_37

In [9]:
item = Sentinel_items[0]
print(item.assets.keys())

dict_keys(['thumbnail', 'overview', 'info', 'metadata', 'visual', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12', 'AOT', 'WVP', 'SCL'])


In [17]:
item.date

datetime.date(2021, 5, 30)

In [ ]:
import os
import boto3
import rasterio as rio
import certifi

os.environ['CURL_CA_BUNDLE'] = certifi.where()
print("Creating AWS Session")
aws_session = rio.session.AWSSession(boto3.Session(), requester_pays=True)


In [ ]:
red_s3 = item.assets['B04']['href']
nir_s3 = item.assets['B08']['href']

In [ ]:
Sentinel_items[0].properties['datetime'][0:10]

In [ ]:
from pyproj import Transformer

def getSubset(geotiff_file, bbox):
    with rio.Env(aws_session):
        with rio.open(geotiff_file) as geo_fp:
            # Calculate pixels with PyProj 
            Transf = Transformer.from_crs("epsg:4326", geo_fp.crs) 
            lat_north, lon_west = Transf.transform(bbox[3], bbox[0])
            lat_south, lon_east = Transf.transform(bbox[1], bbox[2]) 
            x_top, y_top = geo_fp.index( lat_north, lon_west )
            x_bottom, y_bottom = geo_fp.index( lat_south, lon_east )
            # Define window in RasterIO
            window = rio.windows.Window.from_slices( ( x_top, x_bottom ), ( y_top, y_bottom ) )                
            # Actual HTTP range request
            subset = geo_fp.read(1, window=window)
    return subset

In [ ]:
def plotNDVI(nir,red,filename):
    ndvi = (nir-red)/(nir+red)
    ndvi[ndvi>1] = 1
    plt.imshow(ndvi)
    plt.savefig(filename)
    plt.close()

In [ ]:
from rasterio.features import bounds
import matplotlib.pyplot as plt

bbox = bounds(geometry)

for i,item in enumerate(Sentinel_items):
    red_s3 = item.assets['B04']['href']
    nir_s3 = item.assets['B08']['href']
    date = item.properties['datetime'][0:10]
    print("Sentinel item number " + str(i) + "/" + str(len(Sentinel_items)) + " " + date)
    red = getSubset(red_s3, bbox)
    nir = getSubset(nir_s3, bbox)
    plotNDVI(nir,red,"sentinel/" + date + "_ndvi.png")
    break

# for i,item in enumerate(Landsat_items):
#     red_s3 = item['assets']['red']['alternate']['s3']['href']
#     nir_s3 = item['assets']['nir08']['alternate']['s3']['href']
#     date = item['properties']['datetime'][0:10]
#     print("Landsat item number " + str(i) + "/" + str(len(Landsat_items)) + " " + date)
#     red = getSubset(red_s3, bbox)
#     nir = getSubset(nir_s3, bbox)
#     plotNDVI(nir,red,"landsat/" + date + "_ndvi.png")
#     break

In [ ]:
import rasterio as rio
from matplotlib.pyplot import imshow
from rasterio.session import AWSSession

aws_session = AWSSession(requester_pays=True)

cog = 's3://usgs-landsat/collection02/level-2/standard/oli-tirs/2020/026/027/LC08_L2SP_026027_20200827_20200906_02_T1/LC08_L2SP_026027_20200827_20200906_02_T1_SR_B2.TIF'


with rio.Env(aws_session):
    with rio.open(cog) as src:
        profile = src.profile
        arr = src.read(1)
imshow(arr)

In [ ]:
# filename = "geojson file path"# read in AOI as a GeoDataFrame

filename = "map.geojson"# read in AOI as a GeoDataFrame
# read in AOI as a GeoDataFrame
import geopandas as gpd
aoi = gpd.read_file(filename)

# import hvplot.pandas to enable DataFrame.hvplot
import hvplot.pandas

# create a function for later reuse
def plot_polygons(data, *args, **kwargs):
    return data.hvplot.paths(*args, geo=True, tiles='OSM', xaxis=None, yaxis=None,
                             frame_width=600, frame_height=600,
                             line_width=3, **kwargs)

plot_polygons(aoi)